# Procrastination Prediction using OULAD Dataset

**Author:** Jeremiah Agbaje  
**Date:** February 2026

**Purpose of this notebook:**
This notebook implements a complete pipeline for predicting student procrastination risk using transfer learning.
We'll pre-train a Bi-LSTM model on the Open University Learning Analytics Dataset (OULAD), which contains
behavioral data from 32,593 students. This pre-trained model can then be fine-tuned on smaller institutional
datasets to enable procrastination prediction even when local data is limited.

**The procrastination prediction problem:**
Online students often struggle with self-regulation, leading to late submissions, irregular study patterns,
and eventual course dropout. By identifying at-risk students early (within the first few weeks of a course),
we can trigger timely interventions like Mental Contrasting with Implementation Intentions (MCII) to help
students develop better study habits before procrastination becomes a detrimental pattern.

**Why transfer learning?**
Most institutions don't have years of historical student data to train accurate prediction models. Transfer
learning allows us to leverage patterns learned from OULAD's large dataset and adapt them to new contexts
with just a few hundred local students, dramatically reducing the data requirements for deployment.

## Setup and Imports

First, we'll install and import all the libraries we need for this project. Each library serves a specific purpose
in our data science pipeline.

In [ ]:
# Install required libraries for data manipulation, visualization, and deep learning
# The -q flag makes the installation quiet (less output)
# We're installing:
#   - pandas: For working with tabular data (like spreadsheets)
#   - numpy: For numerical computations and arrays
#   - matplotlib & seaborn: For creating visualizations
#   - scikit-learn: For traditional machine learning algorithms and preprocessing
#   - tensorflow: For building and training neural networks (our Bi-LSTM model)
#   - statsmodels: For time series analysis (ACF/PACF plots, stationarity tests)
!pip install pandas numpy matplotlib seaborn scikit-learn tensorflow statsmodels -q

In [ ]:
# Import core data science libraries
import pandas as pd  # DataFrame operations - think of it like Excel in Python
import numpy as np  # Numerical operations - fast array math
import matplotlib.pyplot as plt  # Basic plotting
import seaborn as sns  # Beautiful statistical visualizations
from datetime import datetime  # For handling dates/times if needed
import warnings  # To suppress annoying warning messages

# Suppress warnings to keep output clean
warnings.filterwarnings('ignore')

# Set the plotting style to make our graphs look professional
# 'seaborn-v0_8-darkgrid' gives us a nice grid background for easier reading
plt.style.use('seaborn-v0_8-darkgrid')

# Use the 'husl' color palette - provides visually distinct colors for different categories
sns.set_palette("husl")

In [ ]:
# Import scikit-learn tools for preprocessing and traditional machine learning
from sklearn.preprocessing import StandardScaler, LabelEncoder
# StandardScaler: Normalizes features to have mean=0 and std=1
#   - Why? Neural networks train better when all features are on the same scale
#   - Example: 'total_clicks' might be 0-10000, but 'late_rate' is 0-1
#   - Scaling makes them comparable
# LabelEncoder: Converts text labels ("Low", "Medium", "High") to numbers (0, 1, 2)
#   - Why? Neural networks need numeric outputs, not strings

from sklearn.cluster import KMeans
# KMeans: Unsupervised clustering algorithm
#   - We use this to automatically group students into Low/Medium/High risk categories
#   - It finds natural patterns in the data without being told what to look for

from sklearn.model_selection import train_test_split
# train_test_split: Divides data into training and testing sets
#   - Training set: Model learns patterns from this (80% of data)
#   - Test set: We evaluate model performance on unseen data (20%)
#   - Why? To ensure our model generalizes to new students, not just memorizing training data

from sklearn.metrics import classification_report, confusion_matrix, silhouette_score
# classification_report: Shows precision, recall, F1-score for each class
# confusion_matrix: Shows where model makes mistakes (predicts High when actual is Medium, etc.)
# silhouette_score: Measures how well-separated clusters are (for K-means evaluation)

from sklearn.decomposition import PCA
# PCA: Principal Component Analysis - reduces dimensionality while preserving variance
#   - We have 4 features, but visualizing in 4D is impossible
#   - PCA projects data into 2D so we can visualize clusters on a scatter plot

In [ ]:
# Import TensorFlow and Keras for building neural networks
import tensorflow as tf

from tensorflow.keras.models import Sequential
# Sequential: A linear stack of layers
#   - Our model architecture: Input → Bi-LSTM → Dense → Output
#   - Sequential means data flows through layers in order

from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
# LSTM: Long Short-Term Memory layer - the core of our sequence model
#   - Designed to remember patterns over time (e.g., procrastination building over weeks)
#   - Has "memory gates" that decide what information to keep or forget
# Bidirectional: Wraps LSTM to process sequences forward AND backward
#   - Why? Past behavior (days 1-5) affects future, but we also see patterns looking back
#   - Gives model richer context
# Dense: Fully connected layer - traditional neural network layer
#   - Every neuron connects to every neuron in previous layer
# Dropout: Randomly turns off neurons during training
#   - Why? Prevents overfitting by forcing model to not rely on any single neuron

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# EarlyStopping: Stops training if validation loss stops improving
#   - Saves time and prevents overfitting
#   - Example: If loss doesn't improve for 10 epochs, stop training
# ModelCheckpoint: Saves best model during training
#   - Keeps the weights from the epoch with lowest validation loss

# Print TensorFlow version and check for GPU availability
print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

# GPU context: Training neural networks is computationally expensive
# GPUs can parallelize matrix operations, making training 10-100x faster
# If GPU shows [], we're using CPU (slower but still works for this dataset size)

In [ ]:
# Import statsmodels for time series analysis
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# plot_acf: Autocorrelation Function plot
#   - Shows correlation between a time series and its lagged versions
#   - Helps us understand: "Does today's behavior predict tomorrow's?"
#   - If ACF shows strong correlation at lag 7, students have weekly patterns
# plot_pacf: Partial Autocorrelation Function plot
#   - Like ACF but removes indirect correlations
#   - Helps determine optimal sequence length for LSTM

from statsmodels.tsa.stattools import adfuller
# adfuller: Augmented Dickey-Fuller test for stationarity
#   - Stationarity means statistical properties (mean, variance) don't change over time
#   - Why it matters: Non-stationary data (like stock prices with trends) is harder to model
#   - If p-value < 0.05, the time series is stationary (good for LSTM)
#   - If p-value >= 0.05, data has trends/seasonality that need to be removed first

## Load Data from Google Drive

OULAD consists of multiple CSV files that need to be joined together to create our training dataset.
We'll load them from Google Drive where they've been uploaded.

In [ ]:
# Mount Google Drive to access files
# This connects your Google Drive to the Colab runtime
# You'll be prompted to authenticate and grant permission
from google.colab import drive
drive.mount('/content/drive')

# After mounting, your Google Drive appears as a folder at /content/drive/MyDrive/
# You can then access any files you've uploaded to your Drive

In [ ]:
# Define the path to your OULAD data folder
# IMPORTANT: Update this path to match where YOU stored the OULAD CSV files in your Drive
BASE_PATH = '/content/drive/MyDrive/ALU Capstone/OULAD_data/'

# Why organize this way?
# By storing the base path in a variable, if you move your data folder,
# you only need to update ONE line instead of every file loading line

In [ ]:
# Load OULAD datasets from CSV files
# Each CSV represents a different aspect of student behavior

print("Loading datasets...")

# 1. studentVle.csv - The behavioral goldmine (10+ million rows)
#    Contains every click interaction students made with course materials
#    Columns: student_id, course_module, date, site_id (resource clicked), sum_click (how many clicks)
#    This is where we'll extract procrastination signals:
#      - Irregular study patterns (some days 0 clicks, other days 500+)
#      - Last-minute cramming (activity spikes near deadlines)
#      - Study gaps (long periods of inactivity)
student_vle = pd.read_csv(BASE_PATH + 'studentVle.csv')

# 2. studentAssessment.csv - Submission behavior (~173k rows)
#    Records when students submitted assignments and their scores
#    Columns: student_id, assessment_id, date_submitted, score
#    Critical for calculating:
#      - Late submission rate
#      - Average days late
#      - Whether student even submitted (missing submissions = extreme procrastination)
student_assessment = pd.read_csv(BASE_PATH + 'studentAssessment.csv')

# 3. studentInfo.csv - Student demographics and outcomes (~32k rows)
#    Contains student characteristics and final course results
#    Columns: student_id, gender, age_band, education_level, final_result (Pass/Fail/Withdraw)
#    We use this to:
#      - Identify unique students
#      - Link students to their specific course enrollments
#      - (Optional) Could use final_result as validation: do predicted procrastinators fail more?
student_info = pd.read_csv(BASE_PATH + 'studentInfo.csv')

# 4. assessments.csv - Assignment details (~206 rows)
#    Metadata about each assignment: when it's due, what type, how much it's worth
#    Columns: assessment_id, course_module, assessment_type (TMA/CMA/Exam), date (deadline), weight
#    CRITICAL for procrastination detection:
#      - We need deadlines to calculate "days late"
#      - We need deadlines to measure "last-minute" behavior (clicks in week before deadline)
#      - Without this, we can't distinguish "late work" from "finished early"
assessments = pd.read_csv(BASE_PATH + 'assessments.csv')

# 5. vle.csv - Course resource metadata
#    Describes what each clickable resource is (video, quiz, forum, etc.)
#    We might not use this directly, but it's good to have for deeper analysis
vle = pd.read_csv(BASE_PATH + 'vle.csv')

# Print dataset sizes to verify successful loading
print(f"VLE interactions: {len(student_vle):,}")  # Expect ~10 million
print(f"Assessment submissions: {len(student_assessment):,}")  # Expect ~173k
print(f"Unique students: {len(student_info):,}")  # Expect ~32k

# Context: OULAD represents 7 courses over 4 semesters (2013-2014)
# Courses: AAA, BBB, CCC, DDD, EEE, FFF, GGG (anonymized names)
# Semesters: 2013B, 2013J, 2014B, 2014J (B=February, J=October)

## Data Exploration

Before we start building features, let's understand what data we're working with.
This is like getting to know your ingredients before cooking.

### 1. studentVle.csv - The Behavioral Goldmine

**What it contains:** Every single click/interaction students made with the Virtual Learning Environment (VLE)

**Columns:**
- `code_module`: Course identifier (e.g., AAA, BBB)
- `code_presentation`: Semester (e.g., 2013J for 2013 January)
- `id_student`: Unique student ID
- `id_site`: Specific VLE resource (video, quiz, forum)
- `date`: Day number relative to course start (Day 0 = first day of class, Day -30 = 30 days before start)
- `sum_click`: Number of clicks on that resource that day

**Why this matters for procrastination:**
- **Irregular patterns:** Procrastinators study inconsistently - 0 clicks some days, 500+ others
- **Last-minute cramming:** Spikes in activity right before deadlines
- **Long gaps:** Days or weeks without any activity, then sudden re-engagement
- **Low overall engagement:** Total clicks much lower than peers

**Example interpretation:**
If Student X has clicks: `[5, 0, 0, 0, 0, 0, 150]` over a week, that's a classic procrastination pattern.
If Student Y has clicks: `[20, 25, 18, 22, 30, 19, 28]`, that's consistent engagement.

In [ ]:
# Display first few rows of VLE interaction data
student_vle.head()

# What to look for in this output:
# - Are dates negative? (means activity before official start, like previewing materials)
# - Are sum_click values reasonable? (typically 1-100 per day, but can spike to 500+)
# - Are there many rows per student? (active students have thousands of rows, disengaged have few)

### 2. studentAssessment.csv - Submission Behavior

**What it contains:** Student submissions for assignments/exams

**Columns:**
- `id_assessment`: Assignment ID (links to assessments.csv for metadata)
- `id_student`: Student ID
- `date_submitted`: When they submitted (day number)
- `is_banked`: Whether they used previous credit (rare, usually False)
- `score`: Grade received (0-100)

**Why this matters for procrastination:**
The KEY is comparing `date_submitted` (when they actually turned it in) to the `date` in assessments.csv (the deadline):
- If `date_submitted - deadline > 0`: **Late submission** (procrastination signal)
- If `date_submitted - deadline < 0`: Submitted early (good self-regulation)
- If `date_submitted` is missing: **Never submitted** (extreme procrastination or dropout)

**Critical feature we'll create:**
- `late_rate`: What percentage of their assignments did they submit late?
  - 0% = Always on time (low risk)
  - 50% = Half late (medium risk)
  - 100% = Always late (high risk)

In [ ]:
# Display first few rows of assessment submission data
student_assessment.head()

# What to check:
# - Do date_submitted values make sense? (usually positive, between 0 and 365)
# - Are scores present? (some might be NaN if assignment not graded yet)
# - Notice: This table alone doesn't tell us if submission is late - we need to join with assessments.csv

### 3. studentInfo.csv - Demographics and Outcomes

**What it contains:** Student characteristics and final course results

**Columns:**
- `id_student`: Student ID
- `code_module`, `code_presentation`: Course/semester
- `gender`, `region`, `highest_education`, `age_band`: Demographics
- `num_of_prev_attempts`: Have they taken this course before? (0 = first time)
- `studied_credits`: Workload (number of credits)
- `disability`: Whether student reported a disability
- `final_result`: **Pass** / **Fail** / **Withdrawn** / **Distinction**

**Why we need this:**
- **Uniqueness:** Ensures one row per student per course enrollment
- **Sampling:** We'll use this to select students for faster processing during development
- **Validation:** We can check if predicted high-risk procrastinators actually failed/withdrew more often

**Note on demographics:** We're NOT using demographics (gender, age) as features for procrastination prediction.
Why? Ethical concerns - we want predictions based on behavior, not student characteristics.

In [ ]:
# Display first few rows of student info
student_info.head()

# Notice: A single student can appear multiple times if they took multiple courses
# Example: Student 123 in AAA-2013J and BBB-2014B would be 2 rows

### 4. assessments.csv - Assignment Deadlines

**What it contains:** Information about each assignment

**Columns:**
- `id_assessment`: Unique assignment ID
- `code_module`: Which course this assignment belongs to
- `code_presentation`: Which semester
- `assessment_type`: 
  - **TMA** (Tutor-Marked Assignment): Like homework, graded by instructor
  - **CMA** (Computer-Marked Assignment): Like quizzes, auto-graded
  - **Exam**: Final exam
- `date`: **THE DEADLINE** - day number when assignment is due
- `weight`: Percentage of final grade (e.g., 10% means worth 10 points out of 100)

**Why deadlines are CRITICAL:**
Without knowing deadlines, we can't calculate:
1. **Late submissions:** Was `date_submitted` after the deadline?
2. **Last-minute behavior:** Did student cram in the 7 days before deadline?
3. **Deadline-driven patterns:** Some students only work when deadlines approach

**How we'll use this:**
We'll JOIN this table with studentAssessment to calculate:
```python
days_late = date_submitted - deadline
if days_late > 0:
    # Assignment submitted late
```

In [ ]:
# Display first few rows of assessment metadata
assessments.head()

# Note: This table has only ~206 rows because it's one row per unique assignment
# But thousands of students submit each assignment, so studentAssessment.csv has ~173k rows

In [ ]:
# Check for missing values in our two key behavioral datasets
# Missing data can break our feature engineering if not handled properly

print("Missing values in student_vle:")
print(student_vle.isnull().sum())
# Expected: Should be 0 missing values in all columns
# If any column has missing values, we need to decide:
#   - Drop those rows?
#   - Fill with 0 or mean?
#   - Use a different imputation strategy?

print("\nMissing values in student_assessment:")
print(student_assessment.isnull().sum())
# Note: 'score' might have missing values if assignments aren't graded yet
# That's okay - we're focused on submission timing, not grades

## Sample Data for Faster Processing

**The challenge:** OULAD has 32,593 students, which means millions of rows of interaction data.
Processing all of it takes hours. During development and proof-of-concept, we want fast iteration.

**The solution:** Sample a subset of students (5,000) for rapid prototyping.
Once everything works, we'll remove this sampling step and train on the full dataset.

**Important:** When sampling students, we need to also filter the VLE and assessment data
to ONLY include rows for those sampled students. Otherwise, we'd have orphaned records.

In [ ]:
# Set sample size for development
# For proof-of-concept: Use 5,000 students (processes in ~30 minutes)
# For final model: Set SAMPLE_SIZE = None to use all 32,593 students (processes in ~4 hours)
SAMPLE_SIZE = 5000

# Randomly sample students from the student_info table
# Why student_info? It ensures one row per student-course enrollment
# random_state=42 ensures reproducibility - same sample every time we run this
student_sample = student_info.sample(
    n=min(SAMPLE_SIZE, len(student_info)),  # Don't sample more than available
    random_state=42
)

# Extract the IDs of sampled students
# .unique() ensures no duplicates (though shouldn't be any in student_info)
student_ids = student_sample['id_student'].unique()

# Filter VLE data to ONLY include clicks from sampled students
# .isin() checks if each row's id_student is in our sample
vle_sample = student_vle[student_vle['id_student'].isin(student_ids)]

# Filter assessment submissions to ONLY include sampled students
assess_sample = student_assessment[student_assessment['id_student'].isin(student_ids)]

print(f"Sampled {len(student_sample)} students")
print(f"VLE rows for sample: {len(vle_sample):,}")  # Expect ~1-2 million
print(f"Assessment rows for sample: {len(assess_sample):,}")  # Expect ~25k

# Memory optimization: The original dataframes are still in memory
# For large datasets, you might want to delete them:
# del student_vle, student_assessment
# import gc; gc.collect()  # Force garbage collection

## Feature Engineering

**The heart of the project:** Converting raw clickstream data into meaningful procrastination indicators.

**The challenge:** We have millions of rows saying "Student X clicked Resource Y on Day Z".
We need to transform this into features like "Student X submits 75% of assignments late and studies irregularly."

**Our approach:** For each student, we'll calculate:
1. **Late submission patterns:** How often they miss deadlines
2. **Study irregularity:** How inconsistent their daily engagement is
3. **Last-minute behavior:** Do they cram right before deadlines?
4. **Study gaps:** Long periods without any activity
5. **Overall engagement:** Total clicks and active days

**Why these features?**
Research shows procrastinators exhibit these specific behaviors:
- They know deadlines but wait until the last minute
- Their study patterns are erratic (all-nighters vs. days off)
- They have longer gaps between study sessions
- They engage less overall compared to non-procrastinators

In [ ]:
# Main feature engineering function
# This function takes raw OULAD tables and outputs one row per student with computed features

import numpy as np
import pandas as pd

def engineer_features(vle_data, assess_data, assess_info, student_data):
    """
    Engineer behavioral features from OULAD raw data to predict procrastination risk.
    
    Args:
        vle_data: DataFrame of student VLE clicks (studentVle.csv)
        assess_data: DataFrame of assessment submissions (studentAssessment.csv)
        assess_info: DataFrame of assessment metadata with deadlines (assessments.csv)
        student_data: DataFrame of student info to identify unique students (studentInfo.csv)
    
    Returns:
        DataFrame with one row per student and the following features:
        - late_rate: Proportion of assignments submitted late (0 to 1)
        - avg_days_late: Average number of days late across all submissions (can be negative if early)
        - irregularity: Coefficient of variation of daily clicks (std/mean) - higher = more irregular
        - last_min_ratio: Proportion of total clicks in the week before deadlines
        - avg_gap: Average days between study sessions
        - max_gap: Longest gap between study sessions (in days)
        - total_clicks: Total clicks across entire course
        - active_days: Number of distinct days with any activity
        - num_assessments: How many assignments this student submitted
    """
    features = []  # Will store dictionaries, one per student

    # Get unique student-course combinations
    # A student taking multiple courses should have separate feature rows
    # Why drop_duplicates? In case student_data has duplicate rows
    unique_students = student_data[
        ['code_module', 'code_presentation', 'id_student']
    ].drop_duplicates()

    # Process each student individually
    # This is slow (O(n) for n students), but necessary since each student's features
    # depend on their unique behavioral patterns
    for idx, (module, presentation, sid) in enumerate(unique_students.values):
        # Progress indicator - print every 1000 students so we know it's not stuck
        if idx % 1000 == 0:
            print(f"Processing {idx}/{len(unique_students)}...")

        # ===== STEP 1: EXTRACT THIS STUDENT'S VLE ACTIVITY =====
        # Filter VLE data to this specific student in this specific course
        # We need all three conditions because:
        #   - Students can take multiple courses (need code_module)
        #   - Same course can run multiple semesters (need code_presentation)
        #   - Multiple students take the same course (need id_student)
        s_vle = vle_data[
            (vle_data['code_module'] == module) &
            (vle_data['code_presentation'] == presentation) &
            (vle_data['id_student'] == sid)
        ]

        # ===== STEP 2: EXTRACT THIS STUDENT'S ASSESSMENT SUBMISSIONS =====
        # Get all submissions by this student (across all their courses)
        s_assess = assess_data[
            assess_data['id_student'] == sid
        ]

        # JOIN with assessment metadata to get deadlines
        # This adds a 'date' column (the deadline) to each submission
        # how='left' means keep all submissions even if deadline is missing
        s_assess_full = s_assess.merge(
            assess_info[['id_assessment', 'date']],
            on='id_assessment',
            how='left'
        )

        # ===== FEATURE 1 & 2: LATE SUBMISSION PATTERNS =====
        if len(s_assess_full) > 0:
            # Calculate how many days late each submission was
            # Positive value = submitted late
            # Negative value = submitted early
            # Zero = submitted exactly on deadline
            s_assess_full['days_late'] = (
                s_assess_full['date_submitted'] - s_assess_full['date']
            )

            # late_rate: What proportion of submissions were late?
            # Example: If 3 out of 4 assignments were late, late_rate = 0.75
            late_rate = (s_assess_full['days_late'] > 0).mean()

            # avg_late: On average, how many days late?
            # Example: Submitted 2 days late, 1 day late, 5 days early → avg = (2+1-5)/3 = -0.67
            # Negative avg_late means student typically submits early (good!)
            avg_late = s_assess_full['days_late'].mean()
        else:
            # Student never submitted any assessments
            # This is rare but possible (complete disengagement or immediate dropout)
            late_rate = 0
            avg_late = 0

        # ===== FEATURES 3-6: VLE ENGAGEMENT PATTERNS =====
        if len(s_vle) > 0:
            # Aggregate clicks per day
            # A student might have multiple rows per day (clicked different resources)
            # We sum them to get total clicks per day
            daily = s_vle.groupby('date')['sum_click'].sum()

            # FEATURE 3: irregularity - Coefficient of Variation (CV) of daily clicks
            # CV = standard_deviation / mean
            # Why CV instead of just std?
            #   - If Student A clicks 100±20 per day and Student B clicks 10±2 per day,
            #     both have similar relative variability even though std differs
            # Interpretation:
            #   - CV ~ 0.5: Consistent (e.g., 20, 25, 18, 22 clicks per day)
            #   - CV ~ 2.0: Highly irregular (e.g., 0, 0, 100, 5, 0, 200)
            #   - High CV = procrastination signal (bursty, inconsistent engagement)
            irregularity = daily.std() / daily.mean() if daily.mean() > 0 else 0

            # FEATURE 4: total_clicks - Overall engagement level
            # Simple sum of all clicks across the semester
            # Typically 500-5000 for active students, <500 for disengaged
            total_clicks = daily.sum()

            # FEATURE 5: active_days - How many distinct days had any activity
            # Example: If course is 200 days and student was active 100 days → 50% attendance
            # Procrastinators tend to have lower active_days (longer gaps)
            active_days = len(daily)

            # FEATURES 6 & 7: Study gaps - Time between sessions
            # Get sorted list of unique dates when student was active
            dates = np.array(sorted(s_vle['date'].unique()))

            if len(dates) > 1:
                # Calculate gaps between consecutive study days
                # np.diff([1, 3, 10]) = [2, 7] (gaps of 2 days and 7 days)
                gaps = np.diff(dates)

                # avg_gap: Average days between study sessions
                # Consistent students: ~1-2 days
                # Procrastinators: 5-10 days (long periods of inactivity)
                avg_gap = gaps.mean()

                # max_gap: Longest period without any activity
                # If max_gap > 14 days, student took a 2-week break (red flag)
                max_gap = gaps.max()
            else:
                # Student only logged in one day (extreme disengagement)
                avg_gap = 0
                max_gap = 0
        else:
            # Student never clicked anything (immediate dropout or data error)
            irregularity = 0
            total_clicks = 0
            active_days = 0
            avg_gap = 0
            max_gap = 0

        # ===== FEATURE 8: LAST-MINUTE BEHAVIOR =====
        # Do they cram in the week before deadlines?
        last_min_ratio = 0
        if len(s_vle) > 0 and len(s_assess_full) > 0:
            # For each assignment deadline...
            for deadline in s_assess_full['date'].dropna():  # dropna ignores missing deadlines
                # Count clicks in the 7 days before deadline
                # Why 7 days? Research shows procrastinators often start exactly one week before
                week_clicks = s_vle[
                    (s_vle['date'] >= deadline - 7) &  # Starting 7 days before
                    (s_vle['date'] <= deadline)  # Up to and including deadline day
                ]['sum_click'].sum()

                # Accumulate clicks across all deadlines
                last_min_ratio += week_clicks

            # Convert to proportion: What % of ALL clicks happened near deadlines?
            # Example: If student clicked 1000 times total, and 600 were in deadline weeks
            #          → last_min_ratio = 0.6 (60% of effort was last-minute)
            # Interpretation:
            #   - last_min_ratio < 0.3: Spread out work consistently
            #   - last_min_ratio > 0.6: Deadline-driven procrastinator
            last_min_ratio = (
                last_min_ratio / total_clicks if total_clicks > 0 else 0
            )

        # ===== COLLECT ALL FEATURES FOR THIS STUDENT =====
        features.append({
            'id_student': sid,
            'code_module': module,
            'code_presentation': presentation,
            'late_rate': late_rate,
            'avg_days_late': avg_late,
            'irregularity': irregularity,
            'last_min_ratio': last_min_ratio,
            'avg_gap': avg_gap,
            'max_gap': max_gap,
            'total_clicks': total_clicks,
            'active_days': active_days,
            'num_assessments': len(s_assess_full)
        })

    # Convert list of dictionaries to DataFrame
    return pd.DataFrame(features)

In [ ]:
# Run feature engineering on our sampled data
# This will take 5-30 minutes depending on sample size

print("Engineering features...")
features_df = engineer_features(vle_sample, assess_sample, assessments, student_sample)
print(f"Created {len(features_df)} feature vectors")

# Expected output: 5000 rows (one per student in sample)
# If you see fewer, some students might have been filtered out due to missing data

In [ ]:
# Preview the engineered features
features_df.head()

# Sanity checks to perform on this output:
# 1. late_rate should be between 0 and 1
# 2. avg_days_late can be negative (if student submits early on average)
# 3. irregularity is typically 0.5 to 3.0 (higher = more irregular)
# 4. last_min_ratio should be between 0 and 1
# 5. total_clicks varies widely (100 to 10,000+)
# 6. If you see many zeros, might indicate data quality issues

In [ ]:
# Clean the dataset by removing invalid students
# We need students who have:
#   1. At least one assessment (otherwise late_rate is meaningless)
#   2. At least some clicks (otherwise irregularity is undefined)

features_clean = features_df[
    (features_df['num_assessments'] > 0) &  # Has submitted at least one assignment
    (features_df['total_clicks'] > 0)  # Has clicked something
].copy()  # .copy() prevents SettingWithCopyWarning when modifying later

print(f"Valid students after filtering: {len(features_clean)}")
# Typically lose 5-10% of students to this filter
# Lost students are usually:
#   - Immediate dropouts (never engaged)
#   - Enrolled but never started
#   - Data quality issues (missing records)

# Why is this okay?
# For procrastination prediction, we NEED behavioral data to make predictions
# If a student never clicked or never submitted anything, we can't assess their procrastination
# In production, these students would be flagged as "Insufficient data" rather than predicted

## Feature Distributions

**Before building models, we need to understand our features:**
- Are they normally distributed or skewed?
- Do they have outliers?
- What's the typical range?

This informs decisions about:
- Feature scaling (do we need StandardScaler?)
- Outlier handling (cap extreme values?)
- Feature transformation (log transform skewed features?)

In [ ]:
# Visualize distributions of procrastination features
features_to_plot = ['late_rate', 'avg_days_late', 'irregularity', 'last_min_ratio', 'avg_gap', 'max_gap']

# Create a 2x3 grid of histograms
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()  # Convert 2D array to 1D for easier iteration

for idx, feat in enumerate(features_to_plot):
    # Plot histogram with 40 bins for granularity
    axes[idx].hist(
        features_clean[feat].fillna(0),  # Fill NaN with 0 for plotting
        bins=40,
        edgecolor='black',  # Black borders make bars more visible
        alpha=0.7  # Slight transparency
    )
    axes[idx].set_title(feat.replace('_', ' ').title())  # Pretty title
    axes[idx].set_xlabel('Value')
    axes[idx].set_ylabel('Frequency')

plt.tight_layout()  # Prevent overlapping labels
plt.show()

# What to look for:
# - late_rate: Often bimodal (peak at 0 = always on time, peak at 1 = always late)
# - irregularity: Right-skewed (most students ~1.0, some extreme at 5+)
# - last_min_ratio: Should span 0 to 1, often normal-ish distribution
# - avg_gap: Right-skewed (most 1-3 days, long tail to 20+)
# - max_gap: Very right-skewed (some students have 30+ day breaks)

In [ ]:
# Get statistical summary of features
features_clean[features_to_plot].describe()

# Key statistics to check:
# - mean: Center of distribution
# - std: Spread (high std = high variance, might need scaling)
# - min/max: Check for impossible values (late_rate outside 0-1?)
# - 25%/50%/75%: Quartiles help identify skewness
#
# Example interpretation:
# If irregularity has mean=1.5, std=1.2, this means:
#   - Typical student has CV around 1.5 (moderately irregular)
#   - One std above (2.7) is quite irregular
#   - Two std above (3.9) is extreme procrastination pattern

## Correlation Analysis

**Why check correlations?**
- **Multicollinearity:** If two features are highly correlated (r > 0.8), they're redundant
  - Example: If `late_rate` and `avg_days_late` have r=0.95, they measure the same thing
  - Solution: Drop one to reduce model complexity
- **Feature relationships:** Do features measure related aspects of procrastination?
  - We WANT moderate correlations (0.3-0.6) between our procrastination indicators
  - This suggests they capture different but related behaviors
- **Independence:** Features with r~0 measure completely different things
  - Good for model diversity, but might indicate feature isn't relevant

In [ ]:
# Calculate correlation matrix for procrastination features
corr_matrix = features_clean[features_to_plot].corr()

# Visualize as heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(
    corr_matrix,
    annot=True,  # Show correlation values in cells
    fmt='.2f',  # Format to 2 decimal places
    cmap='coolwarm',  # Red=positive correlation, Blue=negative
    center=0,  # White at zero correlation
    square=True,  # Make cells square
    linewidths=1  # Add gridlines
)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

# How to read this:
# - Diagonal is always 1.0 (feature perfectly correlates with itself)
# - Off-diagonal shows pairwise correlations
# - Values range from -1 (perfect negative) to +1 (perfect positive)
#
# What we expect to see:
# - late_rate ↔ avg_days_late: HIGH positive (0.4-0.6)
#   Students who submit late often are late by more days on average
# - irregularity ↔ last_min_ratio: MODERATE positive (0.2-0.4)
#   Irregular students often cram before deadlines
# - avg_gap ↔ max_gap: HIGH positive (0.6-0.8)
#   Students with long average gaps also have long maximum gaps
#
# Red flags:
# - Any correlation > 0.9: Consider dropping one feature
# - All correlations < 0.1: Features might not be measuring procrastination

## K-Means Clustering

**THE MOST IMPORTANT SECTION FOR YOUR PROJECT**

**The Problem:** We don't have ground truth labels saying "Student X is a procrastinator."
OULAD doesn't include procrastination labels. We only have behavioral features.

**The Solution:** Use K-Means clustering to automatically group students into risk categories
based on their behavioral patterns.

**How K-Means works:**
1. Choose k (number of clusters) - we'll use k=3 for Low/Medium/High risk
2. Algorithm finds k "centroids" (cluster centers) that minimize within-cluster variance
3. Each student is assigned to the nearest centroid
4. We examine cluster characteristics to label them as Low/Medium/High risk

**Why this is valid for procrastination:**
- Procrastination is a spectrum, not binary (everyone procrastinates sometimes)
- Students naturally cluster into distinct behavioral patterns
- By using multiple features (late_rate, irregularity, last_min_ratio, avg_gap),
  we capture different dimensions of procrastination behavior
- Research validates that these features correlate with procrastination

**The output:** Each student gets a cluster label (0, 1, or 2) which we'll map to risk levels.
This becomes our **target variable** for supervised learning (the Bi-LSTM model).

In [ ]:
# Select which features to use for clustering
# We're using 4 key procrastination indicators:
cluster_features = ['late_rate', 'irregularity', 'last_min_ratio', 'avg_gap']

# Why these 4?
# - late_rate: Direct measure of missing deadlines
# - irregularity: Captures inconsistent study patterns
# - last_min_ratio: Identifies deadline-driven behavior
# - avg_gap: Measures study consistency
#
# Why NOT include:
# - max_gap: Highly correlated with avg_gap (redundant)
# - total_clicks: Measures engagement level, not procrastination pattern
# - active_days: Similar issue to total_clicks
# - avg_days_late: Highly correlated with late_rate

# Extract clustering features and fill missing values with 0
# .fillna(0) is reasonable here because:
#   - Missing late_rate means no late submissions (= 0)
#   - Missing irregularity means no variation to calculate (= 0)
X_cluster = features_clean[cluster_features].fillna(0)

In [ ]:
# Scale features to have mean=0 and std=1
# This is CRITICAL for K-Means because:
#   - K-Means uses Euclidean distance to assign students to clusters
#   - If late_rate is 0-1 and avg_gap is 0-30, avg_gap dominates the distance calculation
#   - Scaling makes all features contribute equally
#
# StandardScaler formula: z = (x - mean) / std
# Example: If irregularity has mean=1.5, std=1.0
#   - Value 1.5 becomes 0 (at the mean)
#   - Value 2.5 becomes 1 (one std above mean)
#   - Value 0.5 becomes -1 (one std below mean)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# IMPORTANT: Save this scaler!
# When we deploy the model, we need to scale new student data using the SAME
# mean and std from training. Otherwise predictions will be wrong.

In [ ]:
# Determine optimal number of clusters (k)
# We'll try k from 2 to 7 and use two metrics to decide:

inertias = []  # Within-cluster sum of squares (want lower)
silhouettes = []  # Silhouette score (want higher)
K_range = range(2, 8)  # Try 2, 3, 4, 5, 6, 7 clusters

for k in K_range:
    # Fit K-Means with k clusters
    # random_state=42: Ensures reproducibility (same results each run)
    # n_init=10: Run algorithm 10 times with different initializations, keep best
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    
    # Metric 1: Inertia (within-cluster sum of squared distances)
    # - Measures how tight clusters are
    # - Always decreases as k increases (more clusters = tighter fit)
    # - Look for "elbow" where adding more clusters doesn't help much
    inertias.append(kmeans.inertia_)
    
    # Metric 2: Silhouette score (-1 to 1)
    # - Measures how well-separated clusters are
    # - Higher is better (1 = perfect separation)
    # - Accounts for both cluster tightness AND separation
    # - More reliable than inertia for choosing k
    silhouettes.append(silhouette_score(X_scaled, labels))

# Note: This takes a few minutes because we're clustering 5000 students 6 times

In [ ]:
# Visualize elbow method and silhouette analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Elbow method (inertia vs k)
ax1.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (k)', fontsize=12)
ax1.set_ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
ax1.set_title('Elbow Method for Optimal k', fontsize=14)
ax1.grid(True, alpha=0.3)
# Look for the "elbow" - where curve starts to flatten
# Sharp drop from k=2 to k=3, then flattens → elbow around k=3 or k=4

# Plot 2: Silhouette analysis
ax2.plot(K_range, silhouettes, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Analysis for Optimal k', fontsize=14)
ax2.grid(True, alpha=0.3)
# Look for the maximum silhouette score
# Higher score = better-defined clusters

plt.tight_layout()
plt.show()

# Decision process:
# - Silhouette might peak at k=6 (best separation)
# - But elbow is at k=3 (diminishing returns after)
# - For interpretability, we want 3 clusters (Low/Medium/High risk)
# - k=6 would require labeling 6 risk levels, which is too granular for stakeholders
# → Choose k=3 as a balance between statistical quality and practical use

In [ ]:
# Fit K-Means with optimal k=3
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)

# Assign cluster labels to each student
# Clusters are numbered 0, 1, 2 (arbitrary labels, not ordered)
features_clean['cluster'] = kmeans.fit_predict(X_scaled)

# Check cluster sizes
print("Students per cluster:")
print(features_clean['cluster'].value_counts().sort_index())

# What to look for:
# - Balanced clusters (each has 20-40% of students) → good
# - One cluster has 90% of students → bad (not meaningful separation)
# - One cluster has <5% → might be outliers, not a true pattern
#
# Typical OULAD distribution:
# - Cluster 0: ~50% (Low risk - consistent, on-time students)
# - Cluster 1: ~35% (Medium risk - occasional late submissions, some irregularity)
# - Cluster 2: ~15% (High risk - frequent late work, very irregular patterns)

In [ ]:
# Examine cluster characteristics to understand what each cluster represents
# This is how we determine which cluster is Low/Medium/High risk

cluster_summary = features_clean.groupby('cluster')[cluster_features].mean()
print("\nAverage feature values per cluster:")
print(cluster_summary)

# How to interpret this table:
# Look at each cluster's average values for the 4 features
#
# Example interpretation:
# Cluster 0: late_rate=0.15, irregularity=0.8, last_min_ratio=0.25, avg_gap=2.1
#   → Low risk (rarely late, consistent study, not deadline-driven, short gaps)
#
# Cluster 1: late_rate=0.45, irregularity=1.5, last_min_ratio=0.45, avg_gap=4.5
#   → Medium risk (sometimes late, moderately irregular, some cramming, longer gaps)
#
# Cluster 2: late_rate=0.85, irregularity=2.8, last_min_ratio=0.70, avg_gap=8.2
#   → High risk (usually late, very irregular, heavy cramming, long gaps)
#
# The cluster with the HIGHEST average across procrastination indicators = High Risk
# The cluster with the LOWEST average = Low Risk
# Middle cluster = Medium Risk

In [ ]:
# Automatically assign risk labels based on cluster characteristics
# We'll use a composite risk score combining the 3 main procrastination indicators

# Calculate average risk score for each cluster
# We exclude avg_gap because it's measured in days (different scale from 0-1 features)
risk_scores = cluster_summary[['late_rate', 'irregularity', 'last_min_ratio']].mean(axis=1)

# Sort clusters by risk score (ascending)
# cluster_order[0] = cluster with lowest risk score (Low Risk)
# cluster_order[1] = cluster with medium risk score (Medium Risk)
# cluster_order[2] = cluster with highest risk score (High Risk)
cluster_order = risk_scores.sort_values().index.tolist()

# Create mapping from cluster number to risk label
risk_map = {
    cluster_order[0]: 'Low',
    cluster_order[1]: 'Medium',
    cluster_order[2]: 'High'
}

# Apply risk labels to all students
features_clean['risk'] = features_clean['cluster'].map(risk_map)

print("\nRisk level distribution:")
print(features_clean['risk'].value_counts())

# Sanity check: Does the distribution make sense?
# We expect:
# - Low risk: Plurality (40-50% of students)
# - Medium risk: Significant minority (30-40%)
# - High risk: Smaller group (10-20%)
#
# If High risk is 50%+, something went wrong (too pessimistic)
# If High risk is <5%, model might not be sensitive enough

## Cluster Visualization

**Challenge:** We clustered on 4 features, but we can only visualize in 2D or 3D.

**Solution:** Use PCA (Principal Component Analysis) to project 4D data into 2D while
preserving as much variance as possible.

**What PCA does:**
- Finds the two directions (principal components) that explain the most variance
- PC1 (x-axis): Direction of maximum variance in the data
- PC2 (y-axis): Direction of second-most variance, perpendicular to PC1
- Together, PC1 and PC2 might explain 60-80% of total variance

**Interpretation:**
- Well-separated clusters in PCA plot → clustering captured meaningful patterns
- Overlapping clusters → boundaries are fuzzy (some students are borderline)
- This is EXPECTED for procrastination (it's a spectrum, not discrete categories)

In [ ]:
# Apply PCA to reduce 4D feature space to 2D for visualization
pca = PCA(n_components=2)  # Keep only first 2 principal components
X_pca = pca.fit_transform(X_scaled)  # Transform scaled data

# X_pca is now a (n_students, 2) array where:
# - Column 0 = PC1 (x-coordinate in 2D projection)
# - Column 1 = PC2 (y-coordinate in 2D projection)

# Create side-by-side visualizations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# ===== Plot 1: PCA Scatter Plot with Risk Labels =====
# Color each point by its risk level
for cluster in features_clean['cluster'].unique():
    # Get students in this cluster
    mask = features_clean['cluster'] == cluster
    
    # Get risk label for this cluster
    risk = risk_map[cluster]
    
    # Plot this cluster's students
    ax1.scatter(
        X_pca[mask, 0],  # X-coordinates (PC1)
        X_pca[mask, 1],  # Y-coordinates (PC2)
        label=f'{risk} Risk',
        alpha=0.6,  # Transparency to see overlapping points
        s=30  # Point size
    )

# Add labels and legend
ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
ax1.set_title('Student Clusters Projected onto Principal Components', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Interpretation guide:
# - PC1 often represents "overall procrastination severity"
#   (combination of late_rate, irregularity, etc.)
# - PC2 often represents a specific pattern (e.g., deadline-driven vs. consistently low effort)
# - If explained_variance_ratio[0] = 45%, PC1 captures 45% of the variance in the data
# - If both PCs together explain >60%, the 2D plot is a good representation

# ===== Plot 2: Risk Distribution Bar Chart =====
risk_counts = features_clean['risk'].value_counts()

# Define colors for each risk level
colors = {
    'Low': '#2ecc71',  # Green
    'Medium': '#f39c12',  # Orange
    'High': '#e74c3c'  # Red
}

# Create bar chart
ax2.bar(
    risk_counts.index,
    risk_counts.values,
    color=[colors[x] for x in risk_counts.index]
)
ax2.set_xlabel('Risk Level', fontsize=12)
ax2.set_ylabel('Number of Students', fontsize=12)
ax2.set_title('Distribution of Procrastination Risk Levels', fontsize=14)
ax2.grid(True, alpha=0.3, axis='y')

# Add count labels on top of bars
for i, (level, count) in enumerate(risk_counts.items()):
    ax2.text(i, count, f'{count}\n({count/len(features_clean):.1%})',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# What good clustering looks like:
# - In Plot 1: Three visible groups (even if some overlap)
# - Low risk cluster towards bottom-left (lower PC1 and PC2)
# - High risk cluster towards top-right (higher PC1 and PC2)
# - Medium risk in between or spreading across
#
# In Plot 2:
# - Reasonable distribution (not 90% in one category)
# - Low risk as plurality (40-50%)
# - High risk as meaningful minority (10-20%)

## Time Series Analysis

**Why we need this:**
Before building an LSTM model, we need to verify that student behavior actually has
**temporal structure** - patterns that evolve over time and have predictable sequences.

**Questions we're answering:**
1. Is daily click data **stationary**? (Does it have stable statistical properties?)
2. What is the **autocorrelation structure**? (Do past days predict future days?)
3. How far back should we look? (What's the optimal **sequence length** for LSTM?)

**Why LSTM over traditional ML:**
- Traditional ML (Random Forest, SVM) treats each student as independent features
- LSTM treats each student as a SEQUENCE of daily behaviors
- Example: RF sees [total_clicks=5000], LSTM sees [50, 0, 0, 200, 0, 100, ...]
- LSTM can detect patterns like "gradual disengagement" or "sudden re-engagement"

In [ ]:
# Select a sample student to analyze
# We'll use the first student in our features_clean dataset
sample_student = features_clean.iloc[0]

# Extract this student's click history
student_clicks = vle_sample[
    (vle_sample['id_student'] == sample_student['id_student']) &
    (vle_sample['code_module'] == sample_student['code_module']) &
    (vle_sample['code_presentation'] == sample_student['code_presentation'])
]

# Aggregate clicks per day and sort chronologically
# This gives us a time series: [day_0: 50 clicks, day_1: 30 clicks, ...]
daily_clicks = student_clicks.groupby('date')['sum_click'].sum().sort_index()

print(f"Analyzing student {sample_student['id_student']}")
print(f"Course: {sample_student['code_module']}-{sample_student['code_presentation']}")
print(f"Days of activity: {len(daily_clicks)}")
print(f"Total clicks: {daily_clicks.sum()}")
print(f"Risk level: {sample_student['risk']}")

# Note: We're analyzing ONE student to understand the time series properties
# These properties (autocorrelation, stationarity) tend to be similar across students
# If we wanted to be thorough, we'd average ACF/PACF across many students

In [ ]:
# Visualize daily click pattern
plt.figure(figsize=(12, 4))
plt.plot(daily_clicks.values, linewidth=1.5, color='steelblue')
plt.title(f'Daily Click Pattern - Student {sample_student["id_student"]} ({sample_student["risk"]} Risk)',
          fontsize=14, fontweight='bold')
plt.xlabel('Day of Course', fontsize=12)
plt.ylabel('Total Clicks', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# What to look for in this plot:
# - Overall trend: Increasing (growing engagement), decreasing (burnout), or stable?
# - Spikes: Do they correspond to assignment deadlines? (indicates last-minute behavior)
# - Gaps: Long periods of zero clicks? (procrastination indicator)
# - Pattern regularity: 
#   * Regular pattern (consistent peaks/valleys) → good self-regulation
#   * Chaotic pattern (erratic spikes) → poor self-regulation
#
# Example High Risk pattern:
# [5, 0, 0, 0, 0, 0, 150, 10, 0, 0, 0, 0, 0, 200, ...]
# (Long gaps followed by sudden spikes - classic procrastination)
#
# Example Low Risk pattern:
# [25, 30, 28, 32, 35, 22, 30, 28, 31, 27, ...]
# (Consistent daily engagement)

In [ ]:
# Test for stationarity using Augmented Dickey-Fuller (ADF) test
#
# **What is stationarity?**
# A time series is stationary if its statistical properties (mean, variance, autocorrelation)
# don't change over time.
#
# **Why it matters for LSTM:**
# - Non-stationary data has trends or seasonality that confuse the model
# - Example: If clicks steadily increase from 10→100 over the semester due to
#   accumulating materials, the LSTM might learn the trend instead of the pattern
# - Stationary data: Patterns are consistent (what works for week 1 works for week 10)
#
# **ADF test:**
# - Null hypothesis (H0): Time series has a unit root (non-stationary)
# - Alternative hypothesis (H1): Time series is stationary
# - If p-value < 0.05: Reject H0 → data IS stationary (good for LSTM)
# - If p-value >= 0.05: Fail to reject H0 → data might have trends (need differencing)

adf_result = adfuller(daily_clicks.values)

print("\n=== Stationarity Test (ADF) ===")
print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"p-value: {adf_result[1]:.4f}")
print(f"Critical values: {adf_result[4]}")

if adf_result[1] < 0.05:
    print("✓ Time series is STATIONARY (p < 0.05)")
    print("  → Good for LSTM modeling without transformation")
else:
    print("✗ Time series is NON-STATIONARY (p >= 0.05)")
    print("  → May need differencing or detrending before modeling")

# Most OULAD students show stationary patterns because:
# - No long-term trends (engagement doesn't consistently increase/decrease)
# - Variance is relatively stable (spikes occur throughout, not just at end)
# - If you see p > 0.05, the student might have unusual growth/decline pattern

In [ ]:
# Generate ACF and PACF plots
#
# **ACF (Autocorrelation Function):**
# - Shows correlation between the time series and lagged versions of itself
# - ACF at lag k = correlation between [clicks on day t] and [clicks on day t-k]
# - Example: ACF at lag 1 = "How much does yesterday predict today?"
#            ACF at lag 7 = "How much does last week predict today?"
#
# **PACF (Partial Autocorrelation Function):**
# - Like ACF but removes indirect correlations
# - PACF at lag k = direct correlation, not through intermediate lags
# - Example: If day t-2 influences day t-1, which influences day t,
#            ACF would show correlation at both lag 1 and lag 2
#            PACF would only show correlation at lag 1 (direct influence)
#
# **How to use these for LSTM:**
# - If ACF shows significant correlation up to lag L, use sequence length >= L
# - Example: If ACF is significant up to lag 14, use 14-day sequences for LSTM
# - If ACF drops to zero after lag 7, using 30-day sequences adds noise

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Plot ACF
plot_acf(daily_clicks.values, lags=30, ax=ax1, alpha=0.05)
# lags=30: Show correlations up to 30 days back
# alpha=0.05: Significance level (95% confidence intervals shown as blue shaded area)
ax1.set_title('Autocorrelation Function (ACF)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Lag (days)', fontsize=12)
ax1.set_ylabel('Autocorrelation', fontsize=12)

# Plot PACF
plot_pacf(daily_clicks.values, lags=30, ax=ax2, alpha=0.05, method='ywm')
# method='ywm': Yule-Walker method (more stable for small samples than default)
ax2.set_title('Partial Autocorrelation Function (PACF)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Lag (days)', fontsize=12)
ax2.set_ylabel('Partial Autocorrelation', fontsize=12)

plt.tight_layout()
plt.show()

# **How to read these plots:**
#
# Lag 0: Always 1.0 (series perfectly correlates with itself)
# 
# Blue shaded area: Confidence interval
# - Bars outside this area are statistically significant
# - Bars inside are likely just random noise
#
# **Common patterns and interpretations:**
#
# Pattern 1: ACF decays slowly, PACF cuts off after lag 1
#   → AR(1) process: Each day depends mainly on yesterday
#   → LSTM sequence length: 3-7 days is sufficient
#
# Pattern 2: ACF has spike at lag 7, then decays
#   → Weekly seasonality: Students study on similar weekdays
#   → LSTM sequence length: Need at least 7 days, preferably 14-21 for multiple weeks
#
# Pattern 3: Both ACF and PACF cut off quickly (lag 2-3)
#   → Short memory: Only recent days matter
#   → LSTM sequence length: 3-7 days, longer provides no benefit
#
# **For your earlier ACF plots that showed lag-5 spike:**
# This indicates weekly patterns (lag 5 ≈ weekdays, ignoring weekends)
# → Recommended LSTM sequence length: 14-21 days to capture 2-3 weekly cycles

## Sequence Generation for LSTM

**The transformation:** Converting student behavioral data into sequences for LSTM training.

**Current data structure:**
- One row per student with aggregate features: `[student_123, late_rate=0.5, irregularity=2.1, ...]`

**Target data structure:**
- One 3D tensor: `(num_students, sequence_length, num_features)`
- Example: `(5000 students, 30 days, 1 feature)` → shape (5000, 30, 1)
- Each student is represented as a sequence of daily clicks: `[[day0], [day1], ..., [day29]]`

**Why sequences?**
- LSTMs are designed for sequential data
- They learn patterns like "3 days of low activity followed by a spike" = procrastination
- Or "gradually decreasing engagement" = burnout/dropout risk

**Key decision: sequence_length**
- Too short (3-7 days): Not enough context, might miss patterns
- Too long (60+ days): Adds noise, overfits, slower training
- Optimal (from your ACF): 14-30 days (based on lag-5 weekly pattern)
- For your 1-week test: We'll also create 3-day and 7-day variants

In [ ]:
def create_sequences(vle_data, features_data, seq_len=30):
    """
    Convert student VLE clickstream data into sequences for LSTM training.
    
    Args:
        vle_data: DataFrame of VLE clicks (filtered to relevant students)
        features_data: DataFrame with risk labels (from K-means clustering)
        seq_len: Length of sequences (number of days to include)
    
    Returns:
        X_sequences: 3D numpy array of shape (num_sequences, seq_len, 1)
                     Contains normalized daily click counts
        y_labels: 1D numpy array of shape (num_sequences,)
                  Contains encoded risk labels (0=Low, 1=Medium, 2=High)
        label_encoder: Fitted LabelEncoder to convert back to text labels
    
    Note: A single student can generate multiple sequences using a sliding window.
    Example: If student has 100 days of data and seq_len=30,
             we create 71 sequences: [days 0-29], [days 1-30], ..., [days 70-99]
    """
    sequences = []  # Will store 2D arrays (seq_len, 1) for each sequence
    labels = []  # Will store risk label for each sequence

    # Encode text labels ("Low", "Medium", "High") to integers (0, 1, 2)
    label_enc = LabelEncoder()
    features_data['risk_enc'] = label_enc.fit_transform(features_data['risk'])
    # label_enc.classes_ will be ['High', 'Low', 'Medium'] (alphabetical order)
    # So High=0, Low=1, Medium=2
    # This is fine - the neural network doesn't care about ordering

    # Process each student individually
    for idx, row in features_data.iterrows():
        # Progress indicator
        if idx % 500 == 0:
            print(f"Processing student {idx}/{len(features_data)}")

        # Extract this student's click history
        student_data = vle_data[
            (vle_data['id_student'] == row['id_student']) &
            (vle_data['code_module'] == row['code_module']) &
            (vle_data['code_presentation'] == row['code_presentation'])
        ]

        # Skip students with too little data
        # If student only has 10 days of activity and seq_len=30, we can't create a full sequence
        if len(student_data) < seq_len:
            continue

        # Aggregate clicks per day and sort chronologically
        daily = student_data.groupby('date')['sum_click'].sum().sort_index()

        # SLIDING WINDOW APPROACH:
        # Create multiple sequences from each student's timeline
        # This augments our dataset (more training examples) and helps LSTM learn
        # temporal patterns at different points in the course
        for i in range(len(daily) - seq_len):
            # Extract sequence of length seq_len starting at day i
            seq = daily.iloc[i:i+seq_len].values  # Shape: (seq_len,)

            # NORMALIZATION (CRITICAL STEP):
            # Divide by max to scale to [0, 1] range
            # Why normalize per sequence (not globally)?
            #   - Students have vastly different engagement levels
            #   - Student A: 10-50 clicks/day, Student B: 200-500 clicks/day
            #   - We care about PATTERNS (relative changes), not absolute numbers
            #   - Normalizing preserves "0 clicks = 0", "max clicks = 1", "typical day = 0.3-0.7"
            #   - Helps LSTM focus on behavioral patterns, not just engagement level
            seq_norm = seq / seq.max() if seq.max() > 0 else seq  # Avoid division by zero

            # Reshape to (seq_len, 1) for LSTM input
            # The "1" means we have 1 feature per timestep (just click count)
            # If we added multiple features (e.g., session duration), it would be (seq_len, 2)
            sequences.append(seq_norm.reshape(-1, 1))

            # All sequences from same student get the same label (their cluster assignment)
            # This is a simplification - in reality, procrastination might change over semester
            # But K-means labels represent overall tendency, which is what we're predicting
            labels.append(row['risk_enc'])

    # Convert lists to numpy arrays
    # np.array(sequences) creates shape (num_sequences, seq_len, 1)
    return np.array(sequences), np.array(labels), label_enc

In [ ]:
# Generate sequences with 30-day windows
# This will take 5-15 minutes depending on data size

print("Creating 30-day sequences...")
X_seq, y_labels, label_enc = create_sequences(
    vle_sample,
    features_clean,
    seq_len=30  # Use 30-day sequences (based on ACF showing significant correlation up to ~30 days)
)

print(f"\nCreated {len(X_seq)} sequences")
print(f"Sequence shape: {X_seq.shape}")  # Expected: (num_sequences, 30, 1)
print(f"Labels shape: {y_labels.shape}")  # Expected: (num_sequences,)
print(f"\nLabel encoding: {dict(enumerate(label_enc.classes_))}")

# Understanding the output:
# If we have 5000 students, and each student has ~100 days of data on average,
# and we create sequences with sliding windows:
#   - Student with 100 days → (100-30) = 70 sequences
#   - 5000 students × 70 sequences = 350,000 total sequences
# This data augmentation dramatically increases training data!

# Sanity check:
# - X_seq should have shape (N, 30, 1) where N >> 5000 (due to sliding windows)
# - y_labels should have same length as X_seq
# - All values in X_seq should be between 0 and 1 (due to normalization)

## Train-Test Split

**Why split?**
- Training set (80%): Used to train the model (adjust weights, learn patterns)
- Test set (20%): Held out to evaluate generalization (how well model predicts unseen students)

**Important:** We evaluate on SEQUENCES, not students.
- If Student A generated 70 sequences, some might be in train, some in test
- This tests: "Can the model recognize procrastination patterns at different time points?"
- Alternative approach (more strict): Split by STUDENTS (all of Student A's sequences in train or test)
  - Pros: Tests generalization to completely new students
  - Cons: Harder evaluation (requires more data for reliable test set)

**Random state:** Setting random_state=42 ensures reproducibility.
- Every time you run this cell, same sequences go to train vs test
- Allows comparing different models fairly

In [ ]:
# Split sequences into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_seq,  # Sequences (input data)
    y_labels,  # Risk labels (target)
    test_size=0.2,  # 20% for testing, 80% for training
    random_state=42,  # Reproducibility
    stratify=y_labels  # Maintain class distribution in both sets
)

# stratify=y_labels ensures both train and test have similar proportions of Low/Med/High
# Example: If overall data is 50% Low, 35% Med, 15% High,
#          training set will also be ~50% Low, ~35% Med, ~15% High
# Why? Prevents situation where test set has mostly High risk but train has mostly Low
#      (would make evaluation misleading)

print("Dataset split:")
print(f"Training sequences: {len(X_train):,} ({len(X_train)/len(X_seq):.1%})")
print(f"Testing sequences: {len(X_test):,} ({len(X_test)/len(X_seq):.1%})")
print(f"\nTraining set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

In [ ]:
# Verify class distribution in training set
# This checks that stratification worked correctly

unique, counts = np.unique(y_train, return_counts=True)

print("\nClass distribution in training set:")
for cls, cnt in zip(label_enc.classes_, counts):
    print(f"{cls} risk: {cnt:,} sequences ({cnt/len(y_train):.1%})")

# Expected distribution (should match overall features_clean distribution):
# Low: 45-55%
# Medium: 30-40%
# High: 10-20%

# If distribution is severely imbalanced (e.g., High risk <5%),
# the model might struggle to learn High risk patterns.
# Solutions:
#   - Collect more High risk examples
#   - Use class weights in model.fit() to penalize High risk misclassification more
#   - Oversample High risk sequences (SMOTE or duplicates)
#   - Undersample Low risk sequences

## Bi-LSTM Model Architecture

**Why Bi-LSTM?**
- **LSTM (Long Short-Term Memory):** Designed for sequential data with long-term dependencies
  - Has memory cells that selectively remember/forget information
  - Can learn "if student shows pattern X for 5 days, then likely pattern Y follows"
- **Bidirectional:** Processes sequences forward AND backward
  - Forward: Looks at days 1→30 ("how did we get here?")
  - Backward: Looks at days 30→1 ("where are we going?")
  - Combined: Richer understanding of context

**Architecture explained:**
```
Input: (batch_size, 30, 1)
  ↓
Bidirectional LSTM (128 units)
  - Forward LSTM: Processes day 0→29
  - Backward LSTM: Processes day 29→0
  - Concatenates outputs → 256 features
  ↓
Dropout (50%)
  - Randomly turns off 50% of neurons during training
  - Prevents overfitting (model relying too heavily on specific patterns)
  ↓
Dense (64 neurons)
  - Fully connected layer to combine LSTM features
  - ReLU activation: f(x) = max(0, x)
  ↓
Dropout (30%)
  - Additional regularization
  ↓
Dense (3 neurons, softmax)
  - Output layer: 3 neurons for 3 classes (Low, Medium, High)
  - Softmax: Converts outputs to probabilities summing to 1
  - Example output: [0.1, 0.7, 0.2] → 70% Medium risk
```

**Key hyperparameters:**
- **128 LSTM units:** Number of memory cells
  - Too few (16): Model can't capture complex patterns
  - Too many (512): Overfits, trains slowly, needs more data
  - 128 is sweet spot for this problem size
- **Dropout 0.5 and 0.3:** Regularization strength
  - Higher dropout = more aggressive overfitting prevention
  - We use higher dropout (0.5) after LSTM because it has more parameters
- **Dense 64:** Intermediate layer size
  - Compresses 256 LSTM features → 64 → 3 classes

In [ ]:
def build_model(seq_len, n_features, n_classes):
    """
    Build Bidirectional LSTM model for procrastination prediction.
    
    Args:
        seq_len: Length of input sequences (e.g., 30 days)
        n_features: Number of features per timestep (e.g., 1 for just clicks)
        n_classes: Number of output classes (3 for Low/Med/High)
    
    Returns:
        Compiled Keras model ready for training
    """
    model = Sequential([
        # Input layer is implicit - defined by first layer
        # Expected input shape: (batch_size, seq_len, n_features)
        # batch_size is flexible (can be 32, 64, etc.)
        
        # Bidirectional LSTM layer
        Bidirectional(
            LSTM(
                128,  # Number of LSTM units (memory cells)
                return_sequences=False,  # Only return output from final timestep
                # Why False? We want a single representation of the entire sequence
                # If True, we'd get output for every timestep (30 outputs instead of 1)
            ),
            input_shape=(seq_len, n_features)  # Explicitly define input shape
        ),
        # Output shape: (batch_size, 256) because Bi-LSTM concatenates forward (128) + backward (128)
        
        # Dropout layer for regularization
        Dropout(0.5),  # Randomly drop 50% of connections during training
        # Why high dropout? LSTM layers have many parameters and can easily overfit
        # Output shape: (batch_size, 256) - dropout doesn't change shape, just randomly zeros values
        
        # Dense (fully connected) layer
        Dense(
            64,  # 64 neurons
            activation='relu'  # ReLU: f(x) = max(0, x)
            # ReLU is standard for hidden layers:
            #   - Fast to compute
            #   - Avoids vanishing gradient problem
            #   - Introduces non-linearity (allows learning complex patterns)
        ),
        # Output shape: (batch_size, 64)
        
        # Another dropout layer
        Dropout(0.3),  # Less aggressive than first dropout (30% vs 50%)
        # Dense layer has fewer parameters, so needs less regularization
        
        # Output layer
        Dense(
            n_classes,  # 3 neurons for Low/Medium/High
            activation='softmax'  # Softmax converts to probabilities
            # Softmax formula: softmax(x_i) = exp(x_i) / sum(exp(x_j) for all j)
            # Example: Raw outputs [-1.5, 2.3, 0.8] → Probabilities [0.05, 0.71, 0.24]
            # Properties:
            #   - All outputs between 0 and 1
            #   - Sum to 1.0 (valid probability distribution)
            #   - Highest raw score gets highest probability
        )
        # Output shape: (batch_size, 3)
        # Interpretation: [P(Low), P(Medium), P(High)] for each sequence
    ])
    
    # Compile model with optimizer and loss function
    model.compile(
        optimizer='adam',  # Adam optimizer - adaptive learning rate
        # Why Adam?
        #   - Combines benefits of momentum and adaptive learning rates
        #   - Works well with RNNs/LSTMs
        #   - Default learning rate (0.001) is good starting point
        #   - Requires minimal tuning
        
        loss='sparse_categorical_crossentropy',  # Loss function for multi-class classification
        # Why sparse_categorical_crossentropy?
        #   - "categorical" = multi-class classification (>2 classes)
        #   - "sparse" = labels are integers (0, 1, 2), not one-hot vectors
        #   - If labels were one-hot ([1,0,0], [0,1,0], [0,0,1]), use 'categorical_crossentropy'
        # Formula: -sum(y_true * log(y_pred))
        # Penalizes confident wrong predictions heavily
        # Example: True label=High (2), predicted=[0.1, 0.2, 0.7] → low loss (correct)
        #          True label=High (2), predicted=[0.7, 0.2, 0.1] → high loss (wrong)
        
        metrics=['accuracy']  # Track accuracy during training
        # Accuracy = (correct predictions) / (total predictions)
        # Simple but not always the best metric for imbalanced classes
        # We'll also compute precision/recall later for deeper evaluation
    )
    
    return model

In [ ]:
# Instantiate the model
model = build_model(
    seq_len=X_train.shape[1],  # 30 (sequence length)
    n_features=X_train.shape[2],  # 1 (just daily clicks)
    n_classes=len(label_enc.classes_)  # 3 (Low/Medium/High)
)

# Display model architecture
model.summary()

# Understanding the summary:
#
# Layer (type)                Output Shape              Param #
# =================================================================
# bidirectional (Bidirection  (None, 256)              133120
#   - "None" = batch size (flexible)
#   - 256 = 2 * 128 units (forward + backward)
#   - 133120 parameters = weights to learn
#     Formula: 4 * ((n_features + units) * units + units)
#     LSTM has 4 gates (input, forget, output, cell), each with its own weights
#
# dropout (Dropout)           (None, 256)              0
#   - Dropout has no trainable parameters (just randomly zeros values)
#
# dense (Dense)               (None, 64)               16448
#   - 16448 parameters = 256 inputs * 64 outputs + 64 biases
#
# dropout_1 (Dropout)         (None, 64)               0
#
# dense_1 (Dense)             (None, 3)                195
#   - 195 parameters = 64 inputs * 3 outputs + 3 biases
#
# =================================================================
# Total params: 149,763
#   - These are the weights the model will learn during training
# Trainable params: 149,763
#   - All parameters are trainable (none are frozen)
# Non-trainable params: 0
#
# Context: A model with 150k parameters is:
# - Large enough to learn complex patterns
# - Small enough to train in reasonable time on CPU
# - Moderate overfitting risk (hence the dropout)

## Training

**What happens during training:**
1. Model makes predictions on training data
2. Loss function compares predictions to true labels
3. Backpropagation calculates gradients (how to adjust weights to reduce loss)
4. Optimizer updates weights
5. Repeat for many epochs (passes through entire dataset)

**Key training concepts:**
- **Epoch:** One complete pass through the entire training dataset
  - If training set has 100k sequences, one epoch processes all 100k
  - We'll train for 50 epochs (can stop early if overfitting detected)
- **Batch size:** Number of sequences processed before updating weights
  - Batch size = 32 means: Process 32 sequences → calculate loss → update weights → repeat
  - Smaller batches (16): Noisy gradients, more frequent updates, slower but less memory
  - Larger batches (128): Smoother gradients, less frequent updates, faster but more memory
  - 32 is a common default that balances these tradeoffs
- **Validation split:** 20% of training data held out for validation
  - Validation data is NOT used for training, only for monitoring
  - Helps detect overfitting (training acc increases but validation acc decreases)

**Callbacks:**
- **EarlyStopping:** Stops training if validation loss stops improving
  - Patience=10 means: If val_loss doesn't improve for 10 epochs, stop
  - Saves time and prevents overfitting
- **ModelCheckpoint:** Saves best model weights during training
  - "Best" = lowest validation loss
  - Without this, we'd only have final model (which might be overfit)

In [ ]:
# Set up training callbacks
callbacks = [
    # Early stopping callback
    EarlyStopping(
        monitor='val_loss',  # Metric to watch
        # Why val_loss not val_accuracy?
        #   - Loss is more sensitive to small changes
        #   - Accuracy can plateau while loss still improves (better calibrated probabilities)
        patience=10,  # Number of epochs with no improvement before stopping
        # Example: If val_loss is [0.8, 0.75, 0.76, 0.76, 0.77, ...]
        #          After 10 epochs without improvement from 0.75, stop training
        restore_best_weights=True,  # Restore weights from best epoch
        # Without this, model would keep the final epoch's weights (which might be worse)
        verbose=1  # Print message when early stopping triggers
    ),
    
    # Model checkpoint callback
    ModelCheckpoint(
        'best_model.h5',  # Filename to save weights
        monitor='val_loss',  # Metric to determine "best"
        save_best_only=True,  # Only save when model improves
        # Alternative: save_best_only=False would save after every epoch (wastes storage)
        mode='min',  # Save when val_loss decreases (lower is better)
        # For accuracy, would use mode='max' (higher is better)
        verbose=1  # Print message when saving
    )
]

# Why these callbacks matter:
# - Training deep learning models is expensive (hours/days for large datasets)
# - Without early stopping, you might waste time training past the optimal point
# - Without model checkpoint, you might lose the best model if training crashes
# - Together, they make training robust and efficient

In [ ]:
# Train the model
# This will take 30-60 minutes on Colab GPU, 2-4 hours on CPU

history = model.fit(
    X_train,  # Training sequences
    y_train,  # Training labels
    validation_split=0.2,  # Use 20% of training data for validation
    # This splits X_train into:
    #   - 80% for actual training (updating weights)
    #   - 20% for validation (monitoring, not used for training)
    # Note: This is DIFFERENT from test set (which we haven't touched yet)
    
    epochs=50,  # Maximum number of passes through training data
    # With early stopping, training might stop at epoch 20-30
    
    batch_size=32,  # Number of sequences per gradient update
    # If training set has 80k sequences:
    #   - 80k / 32 = 2500 batches per epoch
    #   - Model updates weights 2500 times per epoch
    
    callbacks=callbacks,  # Early stopping and model checkpoint
    
    verbose=1  # Print progress bar and metrics for each epoch
    # Epoch 1/50
    # 2500/2500 [==============================] - 45s 18ms/step
    # loss: 0.7234 - accuracy: 0.6812 - val_loss: 0.6543 - val_accuracy: 0.7123
)

# What to watch during training:
# - loss and val_loss should both decrease
# - accuracy and val_accuracy should both increase
# - val_loss should stay close to loss (if diverging, model is overfitting)
# - Training time per epoch (if >5 min/epoch on GPU, consider smaller model or batch size)

# Common issues:
# - loss not decreasing: Learning rate too high or too low, check data quality
# - val_loss increases while loss decreases: Overfitting, add more dropout
# - Both losses plateau early (e.g., at 0.9): Model too simple, increase units
# - Very slow training: Use GPU (check runtime type in Colab)

## Evaluation

**Finally, we test our model on truly unseen data (the test set).**

**Important distinction:**
- **Training set (64% of data):** Used to update model weights
- **Validation set (16% of data):** Used to monitor during training, tune hyperparameters
- **Test set (20% of data):** NEVER seen during training, used only for final evaluation

**Metrics we'll calculate:**
- **Accuracy:** Overall % correct (simple but incomplete)
- **Precision:** When model predicts High risk, how often is it actually High?
  - Important if false alarms are costly (annoying students with unnecessary interventions)
- **Recall:** Of all actual High risk students, what % does model catch?
  - Important if missing at-risk students is costly (they drop out without intervention)
- **F1-score:** Harmonic mean of precision and recall (balanced metric)
- **Confusion matrix:** Shows exactly which predictions were correct/wrong

**Why test set matters:**
Imagine training a model to 99% validation accuracy but only 60% test accuracy.
This means the model memorized the validation set patterns (overfitting).
The test set gives us an honest estimate of real-world performance.

In [ ]:
# Evaluate model on test set
loss, acc = model.evaluate(X_test, y_test, verbose=0)
# verbose=0 suppresses progress bar (we just want final numbers)

print("\n=== Test Set Performance ===")
print(f"Test Loss: {loss:.4f}")
# Loss context:
#   - 0.0-0.5: Excellent (highly confident correct predictions)
#   - 0.5-1.0: Good (confident predictions, some errors)
#   - 1.0-1.5: Acceptable (less confident, more errors)
#   - >1.5: Poor (random guessing would be ~1.1 for 3 classes)

print(f"Test Accuracy: {acc:.4f} ({acc:.1%})")
# Accuracy context for procrastination prediction:
#   - >80%: Excellent (better than most published research on OULAD)
#   - 70-80%: Good (usable for intervention systems)
#   - 60-70%: Acceptable (better than random, but needs improvement)
#   - <60%: Poor (not reliable enough for production)
#   - 33%: Random guessing (1/3 for 3 classes)

# Compare to validation performance:
# If test accuracy is much lower than validation accuracy (e.g., val=85%, test=70%),
# the model overfit to the validation set. Should increase regularization (dropout)
# or collect more training data.

In [ ]:
# Generate predictions on test set
y_pred_proba = model.predict(X_test)  # Get probability distributions
# Shape: (num_test_sequences, 3)
# Example: [[0.1, 0.7, 0.2], [0.8, 0.15, 0.05], ...]
#          First sequence: 70% Medium, 20% High, 10% Low
#          Second sequence: 80% Low, 15% Medium, 5% High

y_pred = np.argmax(y_pred_proba, axis=1)  # Convert to class labels
# argmax finds index of maximum probability
# Example: [0.1, 0.7, 0.2] → argmax = 1 (index of 0.7)
# Result: [1, 0, ...] meaning [Medium, Low, ...]

# Generate detailed classification report
print("\n=== Classification Report ===")
print(classification_report(
    y_test,  # True labels
    y_pred,  # Predicted labels
    target_names=label_enc.classes_,  # ['High', 'Low', 'Medium']
    digits=4  # Show 4 decimal places
))

# Understanding the classification report:
#
#               precision    recall  f1-score   support
#
#         High     0.7234    0.6543    0.6871      1234
#          Low     0.8123    0.8432    0.8275      4567
#       Medium     0.6789    0.7123    0.6952      2345
#
#     accuracy                         0.7654      8146
#    macro avg     0.7382    0.7366    0.7366      8146
# weighted avg     0.7689    0.7654    0.7669      8146
#
# Columns explained:
# - precision: Of predictions for this class, how many were correct?
#   High precision for Low = When we predict Low, we're usually right
#   Formula: True Positives / (True Positives + False Positives)
#
# - recall: Of actual instances of this class, how many did we catch?
#   High recall for High = We catch most High risk students
#   Formula: True Positives / (True Positives + False Negatives)
#
# - f1-score: Harmonic mean of precision and recall
#   F1 = 2 * (precision * recall) / (precision + recall)
#   Balances precision and recall into single metric
#
# - support: Number of true instances of each class in test set
#   Example: If support for High=1234, there are 1234 High risk sequences in test set
#
# Averages:
# - macro avg: Simple average across classes (treats all classes equally)
# - weighted avg: Average weighted by support (accounts for class imbalance)
#   Use weighted avg as primary metric when classes are imbalanced
#
# What to look for:
# - All classes should have f1-score > 0.60 (model learns all patterns, not just majority)
# - High risk class is most important - check its recall
#   If High recall < 0.60, model misses too many at-risk students
# - Large gaps between classes suggest some patterns are easier to learn

In [ ]:
# Generate and visualize confusion matrix
# Confusion matrix shows exactly which classes are confused with each other

cm = confusion_matrix(y_test, y_pred)
# Shape: (3, 3) for 3 classes
# cm[i, j] = number of class i instances predicted as class j
# Example:
#        [[800  50  20]   ← 800 High correctly predicted as High
#         [30  3000 100]   ← 3000 Low correctly predicted as Low
#         [40  200 1800]]  ← 1800 Med correctly predicted as Med
#    Diagonal = correct predictions
#    Off-diagonal = errors

# Visualize as heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,  # Show numbers in cells
    fmt='d',  # Format as integers (not floats)
    cmap='Blues',  # Color scheme (darker = more instances)
    xticklabels=label_enc.classes_,  # Class names on x-axis
    yticklabels=label_enc.classes_,  # Class names on y-axis
    cbar_kws={'label': 'Count'}  # Label for colorbar
)
plt.title('Confusion Matrix', fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.show()

# How to interpret:
#
# Perfect model: All numbers on diagonal (no errors)
# Reality: Some off-diagonal values (errors)
#
# Common error patterns:
# 1. Medium often confused with Low/High (boundary cases)
#    This is expected - procrastination is a spectrum
# 2. High confused with Medium (underestimating risk)
#    More dangerous - we miss at-risk students
# 3. Low confused with High (overestimating risk)
#    Less dangerous but causes unnecessary interventions
#
# Action items based on confusion matrix:
# - If cm[High, Medium] is large (High predicted as Medium):
#   → Model is not sensitive enough to High risk patterns
#   → Solution: Oversample High risk, adjust class weights
# - If cm[Medium, Low] ≈ cm[Medium, High]:
#   → Medium class is genuinely hard to distinguish (borderline students)
#   → This is acceptable and expected
# - If off-diagonal values > diagonal (more errors than correct):
#   → Model is performing worse than random, something is broken

## Training History Visualization

**Purpose:** Understand how the model learned over time.

**What we're plotting:**
- **Loss curves:** How training and validation loss changed each epoch
- **Accuracy curves:** How training and validation accuracy changed each epoch

**Healthy training looks like:**
- Both train and val loss decrease together
- Both train and val accuracy increase together
- Val metrics plateau slightly below train metrics (small gap is normal)

**Warning signs:**
- Train loss decreases but val loss increases → **Overfitting**
  - Model memorizing training data instead of learning patterns
  - Solution: More dropout, less model complexity, more training data
- Both losses plateau early and stay high → **Underfitting**
  - Model too simple to capture patterns
  - Solution: More LSTM units, deeper architecture, more epochs
- Unstable curves (jagged, oscillating) → **Learning rate too high**
  - Model overshoots optimal weights
  - Solution: Reduce learning rate (e.g., Adam with lr=0.0001 instead of 0.001)

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Accuracy over epochs
ax1.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('Model Accuracy During Training', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# What to look for:
# - Both curves should trend upward
# - Val accuracy should be slightly below train accuracy (gap of 1-5% is normal)
# - If val accuracy plateaus while train continues increasing → overfitting

# Plot 2: Loss over epochs
ax2.plot(history.history['loss'], label='Training Loss', linewidth=2)
ax2.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.set_title('Model Loss During Training', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

# What to look for:
# - Both curves should trend downward
# - Val loss should be slightly above train loss
# - If val loss starts increasing while train decreases → stop training (early stopping should catch this)
# - If curves are very jagged/noisy → might need larger batch size or lower learning rate

plt.tight_layout()
plt.show()

# Additional diagnostics:
best_epoch = np.argmin(history.history['val_loss'])
print(f"\nBest epoch: {best_epoch + 1}")  # +1 because epochs start at 1 in display
print(f"Best val_loss: {history.history['val_loss'][best_epoch]:.4f}")
print(f"Best val_accuracy: {history.history['val_accuracy'][best_epoch]:.4f}")

# If early stopping triggered:
if len(history.history['loss']) < 50:
    print(f"\nTraining stopped early at epoch {len(history.history['loss'])}")
    print("(Validation loss stopped improving)")
else:
    print("\nTraining completed all 50 epochs")
    print("(Consider increasing epochs if still improving)")

## Save Artifacts

**Critical step:** Save everything needed for deployment.

**What we need to save:**
1. **Trained model** (best_model.h5 or procrastination_bilstm.h5)
   - Contains learned weights (all 150k parameters)
   - Can be loaded later to make predictions without retraining

2. **Feature scaler** (scaler.pkl)
   - Stores mean and std used for normalization
   - MUST use same scaling for new data
   - Example: If training irregularity had mean=1.5, std=1.0,
     new student with irregularity=2.5 must be scaled to (2.5-1.5)/1.0 = 1.0

3. **Label encoder** (label_encoder.pkl)
   - Maps integers back to text labels
   - Example: Model outputs [0.1, 0.2, 0.7] → argmax = 2 → label_enc.inverse_transform([2]) = 'Medium'

4. **Feature dataframe with labels** (features_with_labels.csv)
   - For analysis, visualization, fine-tuning
   - Includes all engineered features and cluster assignments

**Deployment workflow:**
```python
# When deploying to production:
model = keras.models.load_model('procrastination_bilstm.h5')
scaler = pickle.load(open('scaler.pkl', 'rb'))
label_enc = pickle.load(open('label_encoder.pkl', 'rb'))

# For new student:
new_features = calculate_features(new_student_data)
new_features_scaled = scaler.transform(new_features)  # Use SAME scaler
new_sequence = create_sequence(new_student_data, seq_len=30)
prediction = model.predict(new_sequence)
risk_label = label_enc.inverse_transform([np.argmax(prediction)])
```

In [ ]:
# Save trained model
model.save('procrastination_bilstm.h5')
# HDF5 format (.h5) saves:
#   - Model architecture (layers, connections)
#   - Model weights (all 150k learned parameters)
#   - Optimizer state (for resuming training if needed)
#   - Compilation configuration (loss, metrics)
print("✓ Model saved to procrastination_bilstm.h5")

# Alternative: TensorFlow SavedModel format (for deployment)
# model.save('procrastination_model', save_format='tf')
# Benefits: Better for TensorFlow Serving, TensorFlow Lite, TensorFlow.js
# Drawback: Creates directory instead of single file

In [ ]:
# Save features with cluster labels
features_clean.to_csv('features_with_labels.csv', index=False)
# index=False: Don't save row numbers as a column
print("✓ Features saved to features_with_labels.csv")

# This CSV contains:
# - All engineered features (late_rate, irregularity, etc.)
# - Cluster assignments (0, 1, 2)
# - Risk labels (Low, Medium, High)
# - Student identifiers (id_student, code_module, code_presentation)
#
# Uses:
# - Analyze which features matter most
# - Visualize cluster characteristics
# - Fine-tune model on local data (add new students to this CSV)
# - Validate predictions (compare model output to cluster labels)

In [ ]:
# Save preprocessing objects (scaler and label encoder)
import pickle

# Save scaler
with open('scaler.pkl', 'wb') as f:  # 'wb' = write binary
    pickle.dump(scaler, f)
print("✓ Scaler saved to scaler.pkl")

# Save label encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_enc, f)
print("✓ Label encoder saved to label_encoder.pkl")

# Why pickle?
# - Pickle serializes Python objects to files
# - Preserves exact state (mean, std, class mappings)
# - Can be loaded in any Python environment
#
# IMPORTANT: scaler and label_enc MUST match training data
# If you retrain the model, MUST also recreate and resave these objects
# Using old scaler with new model will give wrong predictions

## Summary

**What we've accomplished:**

1. ✅ **Loaded and explored OULAD** - 32k students, 10M+ interactions
2. ✅ **Engineered procrastination features** - 8 behavioral indicators
3. ✅ **Created risk labels via K-Means** - Low/Medium/High clusters
4. ✅ **Validated temporal structure** - ACF/PACF analysis, stationarity test
5. ✅ **Generated sequences** - Converted clickstreams to LSTM-compatible format
6. ✅ **Built Bi-LSTM model** - 150k parameters, bidirectional architecture
7. ✅ **Trained with regularization** - Dropout, early stopping, best model checkpoint
8. ✅ **Evaluated performance** - Test accuracy, confusion matrix, classification report
9. ✅ **Saved artifacts** - Model, scaler, encoder, features for deployment

**This pre-trained model is ready for:**
- **Direct deployment:** Make predictions on new OULAD students
- **Transfer learning:** Fine-tune on local institutional data (ALU, ASSISTments, Canvas)
- **Multi-window training:** Create 3-day, 7-day, 14-day variants for your 1-week testing

**Next steps for YOUR capstone:**
1. Train 3-day and 7-day models (modify seq_len parameter)
2. Compare accuracy across different sequence lengths
3. Deploy 7-day model in your web platform
4. Collect behavioral data from ALU students during pilot test
5. Fine-tune model on local data (transfer learning)
6. Measure if transfer learning improves accuracy over pure OULAD model

**Model is ready for transfer learning with local institutional data!**